In [1]:
import json
import os

In [2]:
json_file = "/home/fer/Escritorio/futstatistics/datasets/dataset6_keypoints/dataset6_keypoints_coco/annotations/train_corrected_annotations.coco.json"
output = "/home/fer/Escritorio/futstatistics/datasets/dataset6_keypoints/dataset6_keypoints_YOLOV5/train/labels_corrected"

In [3]:


class COCO2YOLO:
    def __init__(self):
        self._check_file_and_dir(json_file, output)
        self.labels = json.load(open(json_file, "r", encoding="utf-8"))
        self.coco_id_name_map = self._categories()
        self.coco_name_list = list(self.coco_id_name_map.values())
        print("total images", len(self.labels["images"]))
        print("total categories", len(self.labels["categories"]))
        print("total labels", len(self.labels["annotations"]))

    def _check_file_and_dir(self, file_path, dir_path):
        if not os.path.exists(file_path):
            raise ValueError("file not found")
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    def _categories(self):
        categories = {}
        for cls in self.labels["categories"]:
            categories[cls["id"]] = cls["name"]
        return categories

    def _load_images_info(self):
        images_info = {}
        for image in self.labels["images"]:
            id = image["id"]
            file_name = image["file_name"]
            if file_name.find("\\") > -1:
                file_name = file_name[file_name.index("\\") + 1 :]
            w = image["width"]
            h = image["height"]
            images_info[id] = (file_name, w, h)

        return images_info

    def _bbox_2_yolo(self, bbox, img_w, img_h):
        x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
        centerx = bbox[0] + w / 2
        centery = bbox[1] + h / 2
        dw = 1 / img_w
        dh = 1 / img_h
        centerx *= dw
        w *= dw
        centery *= dh
        h *= dh
        return centerx, centery, w, h

    def _convert_anno(self, images_info):
        anno_dict = dict()
        for anno in self.labels["annotations"]:
            bbox = anno["bbox"]
            image_id = anno["image_id"]
            category_id = anno["category_id"]

            image_info = images_info.get(image_id)
            image_name = image_info[0]
            img_w = image_info[1]
            img_h = image_info[2]
            yolo_box = self._bbox_2_yolo(bbox, img_w, img_h)

            anno_info = (image_name, category_id, yolo_box)
            anno_infos = anno_dict.get(image_id)
            if not anno_infos:
                anno_dict[image_id] = [anno_info]
            else:
                anno_infos.append(anno_info)
                anno_dict[image_id] = anno_infos
        return anno_dict

    def save_classes(self):
        sorted_classes = list(
            map(
                lambda x: x["name"],
                sorted(self.labels["categories"], key=lambda x: x["id"]),
            )
        )
        print("coco names", sorted_classes)
        with open("coco.names", "w", encoding="utf-8") as f:
            for cls in sorted_classes:
                f.write(cls + "\n")
        f.close()

    def coco2yolo(self):
        print("loading image info...")
        images_info = self._load_images_info()
        print("loading done, total images", len(images_info))

        print("start converting...")
        anno_dict = self._convert_anno(images_info)
        print("converting done, total labels", len(anno_dict))

        print("saving txt file...")
        self._save_txt(anno_dict)
        print("saving done")

    def _save_txt(self, anno_dict):
        for k, v in anno_dict.items():
            file_name = os.path.splitext(v[0][0])[0] + ".txt"
            with open(os.path.join(output, file_name), "w", encoding="utf-8") as f:
                print(k, v)
                for obj in v:
                    cat_name = self.coco_id_name_map.get(obj[1])
                    category_id = self.coco_name_list.index(cat_name)
                    box = ["{:.6f}".format(x) for x in obj[2]]
                    box = " ".join(box)
                    line = str(category_id) + " " + box
                    f.write(line + "\n")



In [4]:

c2y = COCO2YOLO()
c2y.coco2yolo()


total images 424
total categories 30
total labels 2934
loading image info...
loading done, total images 424
start converting...
converting done, total labels 424
saving txt file...
0 [('img_L1_79_jpg.rf.f44266bb1a9791aab8e66435e679c078.jpg', 12, (0.6875, 0.6843750000000001, 0.03125, 0.03125)), ('img_L1_79_jpg.rf.f44266bb1a9791aab8e66435e679c078.jpg', 10, (0.465625, 0.19687500000000002, 0.03125, 0.03125)), ('img_L1_79_jpg.rf.f44266bb1a9791aab8e66435e679c078.jpg', 9, (0.76875, 0.178125, 0.03125, 0.03125))]
1 [('98_jpg.rf.ed4f2fb113f0f87d754818096651d3a4.jpg', 18, (0.578125, 0.80625, 0.03125, 0.03125)), ('98_jpg.rf.ed4f2fb113f0f87d754818096651d3a4.jpg', 17, (0.8406250000000001, 0.59375, 0.03125, 0.03125)), ('98_jpg.rf.ed4f2fb113f0f87d754818096651d3a4.jpg', 26, (0.09375, 0.753125, 0.03125, 0.03125)), ('98_jpg.rf.ed4f2fb113f0f87d754818096651d3a4.jpg', 24, (0.47500000000000003, 0.50625, 0.03125, 0.03125)), ('98_jpg.rf.ed4f2fb113f0f87d754818096651d3a4.jpg', 23, (0.30312500000000003, 0.4718750